In [5]:
import requests
from bs4 import BeautifulSoup
import re

import pandas as pd

def recuperer_contenu(url, nom_classe):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Erreur lors de la requête : {e}")
        return []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    balises = soup.find_all(class_=nom_classe)
    
    return balises

In [6]:
# Recuperation des url 

import json
def read_json_file(file_path):
    try:
        # Ouverture et lecture du fichier JSON
        with open(file_path, 'r', encoding='utf-8') as file:
            # Conversion du contenu JSON en dictionnaire Python
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"Erreur: Le fichier {file_path} n'existe pas")
        return None
    except json.JSONDecodeError:
        print(f"Erreur: Le fichier {file_path} n'est pas un JSON valide")
        return None
    except Exception as e:
        print(f"Une erreur inattendue s'est produite: {str(e)}")
        return None

def get_ship(data, name):
    return data[name]
    

def get_url(data,name):
    url = get_ship(data,name)['pledge_url']
    return url


data = read_json_file('shipDB_All.json')

# creation d une liste contenant toutes les url 
 
def list_ship(data):
    Flight_ready=[]
    In_Concept = []
    for i in data:
        if 'pledge_url' in data[i].keys():
            Flight_ready.append(i)
        else : 
            In_Concept.append(i)
    return Flight_ready,In_Concept 
    

In [7]:
Flight_ready = [get_ship(data,i)['pledge_url']  for i in list_ship(data)[0]] 

In [107]:
from collections import defaultdict

def fusion_complete(l):
    lst = lambda:list(range(0,0))
    resultat = defaultdict(lst)
    for d in l:
        for cle, valeur in d.items():
            resultat[cle].append(valeur)

    return dict(resultat)

In [108]:
#creation du dataframe

def DataBase():
    nomclasse = "l-sheet__block"
    dict = {}
    liste_dict=[]
    for i in Flight_ready[:5]:
        dict = {}
        for e in recuperer_contenu(i, nomclasse):
            e = e.text.replace("\n", "|").replace("\t", "")
            e = e.split("|")
            n = []
            for i in e:
                i = re.sub(r'\s+', ' ', i).strip()
                if len(i)!=0:
                    n.append(i)
            for i in range(0, len(n)-2, 2):
                dict[n[i]] = n[i+1]
        print(dict)
        #dict=fusioncomplete(dict,dict)
        liste_dict.append(dict)
    dict=fusion_complete( liste_dict)

    return dict

DB = DataBase()

Erreur lors de la requête : HTTPSConnectionPool(host='robertsspaceindustries.com', port=443): Max retries exceeded with url: /pledge/ships/origin-100/100i (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')))
{}
Erreur lors de la requête : HTTPSConnectionPool(host='robertsspaceindustries.com', port=443): Max retries exceeded with url: /pledge/ships/origin-100/125a (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')))
{}
Erreur lors de la requête : HTTPSConnectionPool(host='robertsspaceindustries.com', port=443): Max retries exceeded with url: /pledge/ships/origin-100/135c (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')))
{}
Erreur lor

In [20]:
DB # probleme sur les valeurs: liste imbriquées

{'Focus': [[[[['Starter / Touring'], 'Starter / Light Fighter'],
    'Starter / Light Freight'],
   'Luxury / Touring'],
  'Pathfinder'],
 'Length': [[[[['19.3m'], '19.3m'], '19.3m'], '27.0m'], '27.0m'],
 'Beam': [[[[['11.0m'], '11.0m'], '11.0m'], '17.0m'], '17.0m'],
 'Height': [[[[['4.0m'], '4.0m'], '4.0m'], '8.0m'], '8.0m'],
 'Size': [[[[['Small'], 'Small'], 'Small'], 'Small'], 'Small'],
 'Cargo Capacity': [[[[['2'], '2'], '6'], '8'], '12'],
 'SCM Speed': [[[[['260 m/s'], '260 m/s'], '260 m/s'], '260 m/s'], '260 m/s'],
 'Afterburner Speed': [[[[['1,425 m/s'], '1,425 m/s'], '1,425 m/s'],
   '1,425 m/s'],
  '1,425 m/s'],
 'Min Crew': [[[[['1'], '1'], '1'], '1'], '1'],
 'Pitch Max': [[[[['- deg/s'], '- deg/s'], '- deg/s'], '- deg/s'], '- deg/s'],
 'Yaw Max': [[[[['- deg/s'], '- deg/s'], '- deg/s'], '- deg/s'], '- deg/s'],
 'Roll Max': [[[[['- deg/s'], '- deg/s'], '- deg/s'], '- deg/s'], '- deg/s'],
 'X-Axis Acceleration': [[[[['- m/s/s'], '- m/s/s'], '- m/s/s'], '- m/s/s'],
  '- m/s/s']

In [21]:
DF = pd.DataFrame(DB)
DF

,Focus,Length,Beam,Height,Size,Cargo Capacity,SCM Speed,Afterburner Speed,Min Crew,Pitch Max,Yaw Max,Roll Max,X-Axis Acceleration,Y-Axis Acceleration
0,"[[[['Starter / Touring'], Starter / Light Figh...","[[[['19.3m'], 19.3m], 19.3m], 27.0m]","[[[['11.0m'], 11.0m], 11.0m], 17.0m]","[[[['4.0m'], 4.0m], 4.0m], 8.0m]","[[[['Small'], Small], Small], Small]","[[[['2'], 2], 6], 8]","[[[['260 m/s'], 260 m/s], 260 m/s], 260 m/s]","[[[['1,425 m/s'], 1,425 m/s], 1,425 m/s], 1,42...","[[[['1'], 1], 1], 1]","[[[['- deg/s'], - deg/s], - deg/s], - deg/s]","[[[['- deg/s'], - deg/s], - deg/s], - deg/s]","[[[['- deg/s'], - deg/s], - deg/s], - deg/s]","[[[['- m/s/s'], - m/s/s], - m/s/s], - m/s/s]","[[[['- m/s/s'], - m/s/s], - m/s/s], - m/s/s]"
1,Pathfinder,27.0m,17.0m,8.0m,Small,12,260 m/s,"1,425 m/s",1,- deg/s,- deg/s,- deg/s,- m/s/s,- m/s/s


In [48]:
vaisseaux =  recuperer_contenu('https://scfocus.org/ship-sale-rental-locations-history/',nom_classe= 'column-1')
prix= recuperer_contenu('https://scfocus.org/ship-sale-rental-locations-history/',nom_classe= 'column-2')

def url_to_list(D):
    L =[]
    for e in D:
        e = e.text.replace("\n", "|").replace("\t", "")
        e = e.split("|")
        L += e
    return L[1:]

prix_doublons= [i for i in zip(url_to_list(vaisseaux),url_to_list(prix))]
prix = dict(set(prix_doublons))

In [ ]:
prix_doublons
copie = prix

def clean_price(price_str):
    # Remove any parentheses and their content
    if '(' in price_str:
        price_str = price_str[:price_str.index('(')].strip()
    
    # Remove commas and convert to float
    values = list(price_str.values())
    for i in values:
        i =  i.replace(',', '')
    return values

# Process the dictionary



AttributeError: 'str' object has no attribute 'values'

In [11]:
recuperer_contenu('https://robertsspaceindustries.com/en/pledge/ships/origin-100/100i',nom_classe='statbox title')

[]

In [71]:
input_prices = prix

def clean_price(price_str):
    # Remove any parentheses and their content
    if '(' in price_str:
        price_str = price_str[:price_str.index('(')].strip()
    
    # Remove commas and convert to float
    return float(price_str.replace(',', ''))

# Process the dictionary
converted_prices = {k: clean_price(v) for k, v in input_prices.items()}
converted_prices

{'Cutter': 635040.0,
 'Constellation Aquila': 12186720.0,
 'Aurora ES': 8467.0,
 'Cyclone MT': 141750.0,
 'Cutlass Black': 61387.0,
 'Valkyrie': 19845000.0,
 'X1 Velocity': 119070.0,
 'Talon': 3260250.0,
 'Mustang Beta': 655200.0,
 'Dragonfly Yellowjacket': 342720.0,
 '125a': 1587600.0,
 'MPUV Cargo': 330750.0,
 '135c': 1490580.0,
 'Constellation Phoenix': 14817600.0,
 'Terrapin': 5433120.0,
 'Avenger Titan': 27165.0,
 'C2 Starlifter': 18900000.0,
 'Mantis': 3402000.0,
 '600i': 816952.0,
 'Nox': 722925.0,
 'Freelancer DUR': 3316950.0,
 'Hull A': 1701000.0,
 'Greycat ROC': 103950.0,
 'Banu Defender': 6237000.0,
 '315p': 1490580.0,
 'MISC Razor EX': 1757700.0,
 'Eclipse': 7938000.0,
 'Reliant Tana': 1984500.0,
 'Ballista': 1481760.0,
 'ALTS': 75600.0,
 'Herald': 1285200.0,
 'Avenger Titan Renegade': 1852200.0,
 'Sabre Firebird': 5874120.0,
 'A2 Hercules Starlifter': 44296876.0,
 'Hornet Wildfire F7C Mk 1': 4630500.0,
 'Corsair': 190008.0,
 'Reclaimer': 31752000.0,
 'X1 Force': 132300.0,


In [72]:
stardle_db = pd.read_json('shipDB_Stardle.json').T
stardle_db = stardle_db.drop('description',axis = 1)
stardle_db

,name,dimensions,cargo_capacity,mass,crew,speed,manufacturer,type,status,role,price,release_date
100i,100i,19,2,48143,1,1425,Origin Jumpworks,multi,flight-ready,"{'0': 'Starter', '1': 'Touring'}",50.0,2020
125a,125a,19,2,48143,1,1425,Origin Jumpworks,multi,flight-ready,"{'0': 'Starter', '1': 'Light Fighter'}",60.0,2020
135c,135c,19,6,48143,1,1425,Origin Jumpworks,multi,flight-ready,"{'0': 'Starter', '1': 'Light Freight'}",65.0,2020
300i,300i,27,8,79677,1,1425,Origin Jumpworks,multi,flight-ready,"{'0': 'Touring', '1': 'Luxury'}",60.0,2014
315p,315p,27,12,80592,1,1425,Origin Jumpworks,exploration,flight-ready,{'0': 'Pathfinder'},65.0,2014
...,...,...,...,...,...,...,...,...,...,...,...,...
X1 Force,X1 Force,1.55,0,4730,1,569,Origin Jumpworks,exploration,flight-ready,{'0': 'Combat'},50.0,2023
X1 Velocity,X1 Velocity,1.55,0,4730,1,611,Origin Jumpworks,competition,flight-ready,{'0': 'Racing'},45.0,2023
Zeus CL Mk II,Zeus CL Mk II,46,128,295000,3,1000,Roberts Space Industries,transport,flight-ready,{'0': 'Medium Freight'},150.0,2024
Zeus ES Mk II,Zeus ES Mk II,46,32,295000,3,1050,Roberts Space Industries,exploration,flight-ready,{'0': 'Expedition'},150.0,2024


In [87]:
stardle_db['name']['Freelancer MAX']

'Freelancer MAX'

In [104]:
stardle_db2 = stardle_db
stardle_db2.shape
#stardle_db2.assign(price = converted_prices.values())
cles = list(converted_prices.keys())
stardle = list(stardle_db2['name'])

def plugin(A,B):
    "A : grande liste B : plus petite liste "
    L=B
    for b in A:
        if b in B.keys():
            L=L
        else :
            L[b]= 42.0
    return L,len(L)

In [105]:
plugin(stardle,converted_prices)

({'Cutter': 635040.0,
  'Constellation Aquila': 12186720.0,
  'Aurora ES': 8467.0,
  'Cyclone MT': 141750.0,
  'Cutlass Black': 61387.0,
  'Valkyrie': 19845000.0,
  'X1 Velocity': 119070.0,
  'Talon': 3260250.0,
  'Mustang Beta': 655200.0,
  'Dragonfly Yellowjacket': 342720.0,
  '125a': 1587600.0,
  'MPUV Cargo': 330750.0,
  '135c': 1490580.0,
  'Constellation Phoenix': 14817600.0,
  'Terrapin': 5433120.0,
  'Avenger Titan': 27165.0,
  'C2 Starlifter': 18900000.0,
  'Mantis': 3402000.0,
  '600i': 816952.0,
  'Nox': 722925.0,
  'Freelancer DUR': 3316950.0,
  'Hull A': 1701000.0,
  'Greycat ROC': 103950.0,
  'Banu Defender': 6237000.0,
  '315p': 1490580.0,
  'MISC Razor EX': 1757700.0,
  'Eclipse': 7938000.0,
  'Reliant Tana': 1984500.0,
  'Ballista': 1481760.0,
  'ALTS': 75600.0,
  'Herald': 1285200.0,
  'Avenger Titan Renegade': 1852200.0,
  'Sabre Firebird': 5874120.0,
  'A2 Hercules Starlifter': 44296876.0,
  'Hornet Wildfire F7C Mk 1': 4630500.0,
  'Corsair': 190008.0,
  'Reclaimer'

In [106]:
stardle_db2.assign(price = plugin(stardle,converted_prices))

ValueError: Length of values (2) does not match length of index (202)